In [2]:
!pip install numpy==1.16.6
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=3408386 sha256=29b543d360e8e7f0a5f89348d09262a08c993c9e1542eee79e492481146f3e02
  Stored in directory: /tmp/pip-ephem-wheel-cache-_4c_3j96/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.3.16 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have 

In [5]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 34.4 MB/s eta 0:00:00


In [3]:
#encoding={'Benign':0, 'FTP-BruteForce':1,'DoS attacks-GoldenEye':2, 'DoS attacks-Slowloris':3,'DoS attacks-SlowHTTPTest':4, 'DoS attacks-Hulk':5,"DDOS attack-HOIC":6,"Bot":7,"DDOS attack-LOIC-UDP":8}

In [6]:
from river import stream
from river import preprocessing
from river import compose
from river import metrics
from river import stats
from river import evaluate
import pandas as pd
from river.feature_selection import PoissonInclusion
from river.imblearn import HardSamplingClassifier
from river.tree import HoeffdingTreeClassifier
from imblearn.metrics import geometric_mean_score
from river.metrics import ClassificationReport

In [7]:
data = pd.read_csv("preprocessed_multiclass_stream.csv")

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [9]:
stream_data = stream.iter_pandas(train.drop("Label", axis = 1), y = train["Label"])

In [10]:
test_data = stream.iter_pandas(test.drop("Label", axis = 1), y = test["Label"])

In [11]:
def construct_pipeline(model):
    return compose.Pipeline(
                PoissonInclusion(p=0.4), # feature selection to reduce training time
                HardSamplingClassifier( # imblearn wrapper to solve class imbalance
                            classifier=model, 
                            p=0.1,
                            size=40,
                        )
                )

In [12]:
HT_model = HoeffdingTreeClassifier()

In [13]:
model=construct_pipeline(HT_model)

In [14]:
for x, y in stream_data:
    model = model.learn_one(x, y)      # make the model learn  

In [15]:
Acc_metric = metrics.Accuracy();
Rec_metric = metrics.Recall();
Pre_metric = metrics.Precision();
F1_metric = metrics.F1();

def get_metric(Acc, rec, pre, f1 ):
  Acc, rec, pre, f1 = Acc, rec, pre, f1
  return([Acc, rec, pre, f1])

predictions = []
true_y = []
for x, y in test_data:
    y_pred = model.predict_one(x) # make a prediction
    predictions.append(y_pred)
    true_y.append(y)
    Acc_metric = Acc_metric.update(y, y_pred)
    Rec_metric = Rec_metric.update(y, y_pred)
    Pre_metric = Pre_metric.update(y, y_pred)
    F1_metric = F1_metric.update(y, y_pred)
    model = model.learn_one(x, y)
print(get_metric(Acc_metric, Rec_metric, Pre_metric, F1_metric))  

[Accuracy: 90.10%, Recall: 87.12%, Precision: 60.47%, F1: 71.39%]


In [16]:
report = ClassificationReport()
for yt,yp in zip(true_y, predictions):
  report = report.update(yt,yp)
print(report)
print("geometric mean score :",geometric_mean_score(true_y, predictions, average='weighted'))

           Precision   Recall   F1       Support  
                                                  
       0      90.94%   96.29%   93.54%      8031  
       1      60.47%   87.12%   71.39%      5224  
       2      85.73%   99.40%   92.06%      8204  
       3      96.30%   92.10%   94.16%      2064  
       4      59.62%   24.57%   34.80%       396  
       5      87.25%   59.12%   70.48%      3217  
       6      99.92%   99.93%   99.93%     17001  
       7      99.60%   95.25%   97.37%     14366  
       8      99.72%   99.72%   99.72%       356  
                                                  
   Macro      86.62%   83.72%   83.72%            
   Micro      90.10%   90.10%   90.10%            
Weighted      90.20%   90.10%   89.25%            

                 90.10% accuracy                  
geometric mean score : 0.9440602609898785
